In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

features_and_target = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_next_27_hour,rides_next_28_hour,rides_next_29_hour,rides_next_30_hour,rides_next_31_hour,rides_next_32_hour,rides_next_33_hour,rides_next_34_hour,rides_next_35_hour,rides_next_36_hour
0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,10.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,4.0,0.0,0.0,0.0,1.0,2.0,5.0,2.0,2.0,2.0
2,0.0,1.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,2.0,0.0,5.0,4.0,1.0,8.0
3,4.0,2.0,2.0,1.0,0.0,0.0,0.0,2.0,1.0,4.0,...,2.0,1.0,1.0,0.0,1.0,1.0,7.0,2.0,2.0,3.0
4,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111211,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
111212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,3.0,0.0
111213,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0
111214,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0


In [3]:
features = features_and_target.filter(regex='previous|pickup')
targets = features_and_target.filter(like='next')

In [4]:
features_ = features.iloc[500]
target_ = targets.iloc[500]
    
ts_columns_features = [c for c in features.columns if c.startswith('rides_previous_')]# or c.startswith('rides_next_')]
ts_columns_targets = [c for c in targets.columns if c.startswith('rides_next_')]
ts_values_features = [features_[c] for c in ts_columns_features] # + target_.values.tolist()
ts_values_targets = [target_[c] for c in ts_columns_targets] 

In [5]:
len(ts_columns_features)

672

In [6]:
+672+36

708

In [7]:
len(ts_values_features)

672

In [8]:
len(ts_columns_features)

672

In [9]:
24*7*4

672

In [10]:
len(ts_values_targets)

36

In [11]:
features_['pickup_hour']

Timestamp('2022-07-12 00:00:00')

In [12]:
from datetime import timedelta
pd.date_range(
        features_['pickup_hour'] - timedelta(hours=len(ts_columns_features)),
        features_['pickup_hour'] - timedelta(hours=1),
        freq='H'
    )

DatetimeIndex(['2022-06-14 00:00:00', '2022-06-14 01:00:00',
               '2022-06-14 02:00:00', '2022-06-14 03:00:00',
               '2022-06-14 04:00:00', '2022-06-14 05:00:00',
               '2022-06-14 06:00:00', '2022-06-14 07:00:00',
               '2022-06-14 08:00:00', '2022-06-14 09:00:00',
               ...
               '2022-07-11 14:00:00', '2022-07-11 15:00:00',
               '2022-07-11 16:00:00', '2022-07-11 17:00:00',
               '2022-07-11 18:00:00', '2022-07-11 19:00:00',
               '2022-07-11 20:00:00', '2022-07-11 21:00:00',
               '2022-07-11 22:00:00', '2022-07-11 23:00:00'],
              dtype='datetime64[ns]', length=672, freq='H')

In [13]:
from datetime import timedelta
ts_dates_features = pd.date_range(
        features_['pickup_hour'] - timedelta(hours=len(ts_columns_features)),
        features_['pickup_hour'] - timedelta(hours=1),
        freq='H'
    )
ts_dates_targets = pd.date_range(
        features_['pickup_hour'],
        features_['pickup_hour'] + timedelta(hours=len(ts_columns_targets)-1),
        freq='H'
    )

In [14]:
len(ts_dates_features)

672

In [15]:
len(ts_dates_targets)

36

In [16]:
import plotly.express as px 
# line plot with past values
title = f'Pick up hour={features_["pickup_hour"]}, location_id={features_["pickup_location_id"]}'
fig = px.line(
        x=ts_dates_features, y=ts_values_features,
        template='plotly_dark',
        markers=True, title=title
    )
# green line for the values we wanna predict
targets_fig = px.line(x=ts_dates_targets, y=target_.values.tolist(),
                template='plotly_dark', 
                markers=True, title='actual values')
targets_fig.update_traces(line_color='green')

fig.add_traces(targets_fig.data)
#fig.show()



In [19]:
from src.plot import plot_one_sample

plot_one_sample(
    features,
    targets,
    example_id=500,
)